# Volume rendering

## Prerequisites
You need to install the following modules to run this notebook
```bash
$conda install -c conda-forge pyvista
$conda install -c conda-forge imageio-ffmpeg
```

## Load modules

In [1]:
import numpy as np
import matplotlib.pyplot as plt # not really needed but handy if you want to look at histogram and slices
import pyvista as pv
import tifffile as tiff

## Load the data

In [2]:
img=tiff.imread('legorecon.tif')

## Prepare data container
PyVista needs to prepare a container that describes the volume. Here, we will connect the numpy array with a Uniform grig

In [3]:
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape + 1 because we want to inject our values on
#   the CELL data
grid.dimensions = np.array(img.shape) + 1

# Edit the spatial reference
grid.origin = (0, 0, 0)   # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis, i.e., the pixel size 

# Add the data values to the cell data
grid.cell_data["values"] = img.flatten(order="F")  # Flatten the array!

/Users/kaestner/anaconda3/lib/python3.11/site-packages/pyvista/core/grid.py:912: PyVistaDeprecationWarning: `UniformGrid` is deprecated. Use `ImageData` instead.
  warnings.warn(


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

## Interactive rendering
This mode allows you to interact with the rendered data using the mouse. You can grab and rotate the volume and other manipulators. 

This is the method you'd use to explore you data.

### Arbitrary cut planes
This can take a while to render, and the interaction is quite slow.

In [5]:
p = pv.Plotter()
p.add_mesh_clip_plane(grid)
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Orthogonal planes
This is a faster option and you can still rotate the scene with the mouse.

In [6]:
slices = grid.slice_orthogonal(x=img.shape[0]//2, z=img.shape[2]//2)
cpos = [
    (540.9115516905358, -617.1912234499737, 180.5084853429126),
    (128.31920055083387, 126.4977720785509, 111.77682599082095),
    (-0.1065160140819035, 0.032750075477590124, 0.9937714884722322),
]
dargs = dict(cmap='plasma')

p = pv.Plotter()
p.add_mesh(slices, **dargs)

p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Change the opaticty

In [7]:

p1 = pv.Plotter()
va = p1.add_volume(img,cmap='viridis')
f = lambda val: va.GetProperty().SetScalarOpacityUnitDistance(val)
p1.add_slider_widget(f, [-0.1, 1], title="Opacity Distance")
p1.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Movies

### Basic example with geometric model
This first example uses a mesh in the rendering.

In [10]:
import numpy as np

import pyvista as pv

filename = "sphere-shrinking.mp4"

mesh = pv.Sphere()
mesh.cell_data["data"] = np.random.random(mesh.n_cells)

plotter = pv.Plotter()
# Open a movie file
plotter.open_movie(filename)

# Add initial mesh
plotter.add_mesh(mesh, scalars="data", clim=[0, 1])
# Add outline for shrinking reference
plotter.add_mesh(mesh.outline_corners())

plotter.show(auto_close=False)  # only necessary for an off-screen movie

# Run through each frame
plotter.write_frame()  # write initial data

# Update scalars on each frame
for i in range(100):
    random_points = np.random.random(mesh.points.shape)
    mesh.points = random_points * 0.01 + mesh.points * 0.99
    mesh.points -= mesh.points.mean(0)
    mesh.cell_data["data"] = np.random.random(mesh.n_cells)
    plotter.add_text(f"Iteration: {i}", name='time-label')
    plotter.write_frame()  # Write this frame

# Be sure to close the plotter when finished
plotter.close()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Orbiting orthoplanes

In [11]:
p = pv.Plotter()
p.set_background('white')
p.add_mesh(slices, lighting=False)
p.camera.zoom(2.5)
p.show(auto_close=False)
path = p.generate_orbital_path(n_points=36, shift=mesh.length)
filename='orthoplanes.mp4'
#p.open_movie(filename)
p.open_gif("orbit.gif")
p.orbit_on_path(path, write_frames=True)
# p.orbit_on_path(path)
p.close()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)